In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np
import itertools

In [2]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,[trend],True,NaN


In [3]:
rankobj = dsp.mean_rank_trend()
linregobj = dsp.linear_trend()

The tren objects above will compute all pairs of given types, but what if we want to define custom trends?  We can do that by overloading existing types.  We'll overload only the get_trend_vars() function for now, but the other functions can also be overloaded or a totally new trend can be added as long as it is compatible.

In [6]:
class min_lin_reg(dsp.linear_trend):
    symmetric_vars = False # tell it not to do combinations
    
    def get_trend_vars(self,labeled_df):
        """
        """
        x_vars = ['year']
        y_vars = ['search_conducted_rate','contraband_found_rate','hit_rate','num_stops','search_conducted_true']
        
        self.regression_vars = list(itertools.product(x_vars,y_vars))
        return self.regression_vars
    
min_lin_reg_obj = min_lin_reg()

In [9]:
for a,b in min_lin_reg_obj.regression_vars:
    print(a)

year
year
year
year
year


In [7]:
labeled_df.get_subgroup_trends_1lev([min_lin_reg_obj,rankobj])
labeled_df.result_df.sample(10)

KeyError: ('year', 'search_conducted_rate')

# Component-wise

We can also use the components of trends to construct custom trends

In [ ]:
from detect_simpsons_paradox import trend_components as tcomp

In [ ]:
# np.repeat(labeled_df.df['search_conducted_rate'].values,labeled_df.df['num_stops'].values)
# labeled_df.df['search_conducted_rate'].values
# np.repeat([1,4,5,2,35],[3,2,4,1,7])


In [ ]:
def w_median(df,mcol,wcol):
    """
    """
    if pd.isna(wcol):
        wmed = df[mcol].median()
    else:
        reps = [int(n) for n in df[wcol].values]
        repd_mcol = np.repeat(df[mcol].values,reps)
        wmed = np.median(df[mcol]*df[wcol])
    return wmed
    
    
class weightedMedianRank(tcomp.weightedMeanRank):
    """
    common parts for all continuous variable trends
    """
    my_stat = w_median

#     def get_trend_vars(self,labeled_df):
#         """
#         """
#         # maybe not counts

#         self.target = labeled_df.get_vars_per_roletype('trend',['binary','continuous'])
#         self.trendgroup = labeled_df.get_vars_per_roletype(['trend','explanatory'],'categorical')
#         self.var_weight_list = labeled_df.get_weightcol_per_var(self.target)
#         return self.target, self.trendgroup

class median_rank_trend(tcomp.rankTrend,weightedMedianRank,tcomp.trend):
    name = 'medain_rank_trend'

In [ ]:
medianrankobj = median_rank_trend()
labeled_df.get_subgroup_trends_1lev([min_lin_reg_obj,rankobj,medianrankobj])
labeled_df.result_df.sample(10)